In [5]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from time import sleep
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as Ec
from requests.packages.urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import re
from selenium.webdriver.chrome.options import Options

In [6]:
path = r'chromedriver.exe'

service = Service(path)

# set up chrome options
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--window-size-1920,1080')

#initialize browser
browser = webdriver.Chrome(service=service, options=options)

In [7]:

headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.4pre) Gecko/20070410 BonEcho/2.0.0.4pre'}


In [8]:
url = 'https://www.99acres.com/commercial-property-for-rent-in-bangalore-ffid?&page=1'

In [9]:
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.content, 'html.parser')
print(r)

<Response [200]>


In [14]:

sites = soup.find('div', class_='tupleNew__outerTupleWrap undefined')    #'tupleNew__innerCont')
print(sites.prettify())

<div class="tupleNew__outerTupleWrap undefined" data-custominfo='{"payload":{"search_results":{"selected_entity_tuples":[{"id":"53097984","rank":1,"res_com":"C","attribute":"FSL","attributes":{"rtov_enabled":"Y","rtov_booked":"","hasOffer":0,"isPreLeased":""}}]}}}' data-label="FSL_TUPLE.1" data-premium="false" data-rank="1_1" id="N53097984" topmost="true">
 <div class="tupleNew__tupleWrapTopaz tupleNew__platHTopaz">
  <div class="tupleNew__innerCont" id="AI_LISTING">
   <div class="tupleNew__imgWrap tupleNew__platinumTopaz" id="tuple_srp_0_vgqiguZQJNA">
    <div class="tupleNew__topLeftItems">
     <div style="display:flex">
      <div>
       <img alt="Featured-Tag" class="ImgItem__tag" height="20" src="https://static.99acres.com/universalapp/img/Featured.png" style="width:63px;height:20px" width="63"/>
      </div>
     </div>
    </div>
    <div class="tupleNew__topRightItemsTopaz">
     <div class="ImgItem__shortlistBtnWrapTopaz" data-custominfo='{"payload":{"search_results":{"sele

In [25]:
link_tag = sites.find('a', class_='tupleNew__propertyHeading')
if link_tag and link_tag.has_attr('href'):
    property_url = link_tag['href']
    print("Extracted URL:", property_url)
    
else:
    print("Link not found.")

Extracted URL: https://www.99acres.com/ready-to-move-office-space-for-rent-lease-in-manyata-tech-park-bangalore-north-14000-sq-ft-r13-spid-N53097984


In [26]:
Rent =link_tag.find('span', id='LeaseAmout')
print(Rent)

None


In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import random
import re
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from urllib.parse import quote

# Set up Chrome driver path
path = r'chromedriver.exe'
service = Service(path)

# Set up Chrome options
options = Options()
options.add_argument("--headless=new")  # Use new headless mode for better stability
options.add_argument("--disable-gpu")  
options.add_argument("--window-size=1920,1080")
options.add_argument("--disable-blink-features=AutomationControlled")  # Bypass bot detection

# Initialize headless browser
browser = webdriver.Chrome(service=service, options=options)

headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.4pre) Gecko/20070410 BonEcho/2.0.0.4pre'}
#headers = {'User-Agent': 'Mozilla/4.0 (compatible; MSIE 6.0; AOL 9.0; Windows NT 5.1; {}; Alawar 2.08; .NET CLR 1.0.3705)'}
#headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}

'''headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}
'''

print('Started fetching the links..............')
# Scraping property links
links = []
for i in range(2):    
    url = f'https://www.99acres.com/commercial-property-for-rent-in-bangalore-ffid?&page={i}'
    try:
        req = requests.get(url, headers =  headers)
        soup = BeautifulSoup(req.content, 'html.parser')
        
        sites = soup.find_all('div', class_='tupleNew__tupleHeadingTopaz')

        for site in sites:
            for link in site.find_all('a', class_='tupleNew__propertyHeading ellipsis', href=True):
                #link = link['href']
                links.append(link['href'])
                #print(link['href'])
   
        
        print(f"Page {i + 1} scraped, total links: {len(links)}")
        sleep(random.uniform(1, 4))  

    except requests.exceptions.RequestException as e:
        print(f"Error fetching page {i + 1}: {e}")
        continue

print(" Starting to extract Data...........")

# Scraping property details
properties = []
for index, link in enumerate(links):
    try:
        r = requests.get(link, headers=headers, timeout=10)
        r.raise_for_status()
        soup2 = BeautifulSoup(r.content, 'html.parser')

        ###  Property Name     Address       Area       Rental price       Property type   Carpet area(sq ft)    Contact     Maps link
        

        property_name = soup2.find('div', class_="component__pdPropConfSide component__rentHeading pd__rentHeading")
        Property_Name = property_name.text.strip() if property_name else 'N/A'
        
        address = soup2.find('span', class_='component__pdPropAddress')
        Address = address.text.strip() if address else 'N/A'
        
        area_tag = soup2.find('span', id='superbuiltupAreaId')
        Area = area_tag.text.strip() if area_tag else 'N/A'
              
        rental_price = soup2.find('span', id='LeaseAmout')
        Rental_Price = rental_price.get_text(strip=True) if rental_price else 'N/A'

        property_type = soup2.find('span', id='landZone')
        Property_Type = property_type.get_text(strip=True) if property_type else 'N/A'

        carpet_area = soup2.find('span', id='carpetAreaId')
        Carpet_Area = carpet_area.get_text(strip=True) if carpet_area else 'N/A'

        contact = soup2.find('', class_='')
        Contact = contact.get_text(strip=True) if contact else 'N/A'

        if Address != 'N/A':
            #encoded_address = urllib.parse_quote_plus(Address)
            encoded_address = quote(Address)
            Maps_Link = f"https://www.google.com/maps/search/?api=1&query={quote(encoded_address)}"
        else:
            Maps_Link = 'N/A'

        #maps_link = soup2.find('', class_="")
        #Maps_Link = maps_link.text.strip() if maps_link else 'N/A'

        
        #print(Title , Location, Area, price, Listed_on)
        details = {
            'Property Name': Property_Name,
            'Address':Address,
            'Area':Area,
            'Rental price':Rental_Price,
            'Property Type': Property_Type,
            'Carpet area(sq ft)' : Carpet_Area ,
            'Contact':  Contact,
            'Maps link': Maps_Link ,
            'url':link
        }
              
        properties.append(details)

        print(f"Done {index + 1}/{len(links)}")

        sleep(random.uniform(1, 5))  
    except Exception as e:
        print(f"Error processing {link}: {e}")
        continue

# Save to CSV
df = pd.DataFrame(properties)
df.to_csv('99ACRE(commercial_space).csv', index=False)
print("Data saved to 99ACRE(commercial_space).csv")


browser.quit()


Started fetching the links..............
Page 1 scraped, total links: 29
Page 2 scraped, total links: 59
 Starting to extract Data...........
Done 1/59
Done 2/59
Done 3/59
Done 4/59
Done 5/59
Done 6/59
Done 7/59
Done 8/59
Done 9/59
Done 10/59
Done 11/59
Done 12/59
Done 13/59
Done 14/59
Done 15/59
Done 16/59
Done 17/59
Done 18/59
Done 19/59
Done 20/59
Done 21/59
Done 22/59
Done 23/59
Done 24/59
Done 25/59
Done 26/59
Done 27/59
Done 28/59
Done 29/59
Done 30/59
Done 31/59
Done 32/59
Done 33/59
Done 34/59
Done 35/59
Done 36/59
Done 37/59
Done 38/59
Done 39/59
Done 40/59
Done 41/59
Done 42/59
Done 43/59
Done 44/59
Done 45/59
Done 46/59
Done 47/59
Done 48/59
Done 49/59
Done 50/59
Done 51/59
Done 52/59
Done 53/59
Done 54/59
Done 55/59
Done 56/59
Done 57/59
Done 58/59
Done 59/59
Data saved to 99ACRE(commercial_space).csv


In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import random
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from urllib.parse import quote
from fake_useragent import UserAgent
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Setup ChromeDriver with anti-bot options
def init_browser():
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument(f"user-agent={UserAgent().random}")
    return webdriver.Chrome(service=Service('chromedriver.exe'), options=chrome_options)

# Generate headers dynamically
def get_headers():
    ua = UserAgent()
    return {
        'User-Agent': ua.random,
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Connection': 'keep-alive'
    }

#URLs using BeautifulSoup
def get_listing_links(pages=2):
    links = []
    for i in range(1, pages + 1):
        url = f'https://www.99acres.com/commercial-property-for-rent-in-bangalore-ffid?&page={i}'
        try:
            logging.info(f"Fetching page {i}")
            resp = requests.get(url, headers=get_headers(), timeout=15)
            soup = BeautifulSoup(resp.content, 'html.parser')
            sites = soup.find_all('div', class_='MobileTuppleWrapper__tupleWrapperStyle pageComponent')

            for site in sites:
                for link in site.find_all('a', class_='resBuy__propertyHeading', href=True):
                    links.append(link['href'])

            sleep(random.uniform(2, 5))  

        except Exception as e:
            logging.warning(f"Failed on page {i}: {e}")
    return links

# Extract property details
def extract_property_data(browser, url):
    try:
        browser.get(url)
        sleep(random.uniform(3, 6)) 
        soup = BeautifulSoup(browser.page_source, 'html.parser')

        prop_name = soup.find('div', class_="component__pdPropConfSide component__rentHeading pd__rentHeading")
        address = soup.find('span', class_='component__pdPropAddress')
        area_tag = soup.find('span', id='superbuiltupAreaId')
        rental_price = soup.find('span', id='LeaseAmout')
        property_type = soup.find('span', id='landZone')
        carpet_area = soup.find('span', id='carpetAreaId')
        
        # Simulated contact and map generation
        Contact = 'Available after login'  # need to work on with

       ''' # Click the Contact Dealer button to reveal phone
        try:
            contact_btn = WebDriverWait(browser, 10).until(
                EC.element_to_be_clickable((By.ID, "contactDealerBtn"))
            )
            browser.execute_script("arguments[0].click();", contact_btn)
            sleep(random.uniform(2, 4))  # allow time for contact to appear
        except Exception as e:
            logging.warning(f"Couldn't click Contact Dealer on {url}: {e}")

        # Extract phone number
        Contact = "N/A"
        try:
            contact_div = browser.find_element(By.CLASS_NAME, "component__activeDetails")
            spans = contact_div.find_elements(By.TAG_NAME, "span")
            if spans:
                Contact = spans[0].text.strip()
        except Exception as e:
            logging.warning(f"Contact number not found on {url}: {e}")
'''
        
        encoded_address = quote(address.text.strip()) if address else ''
        Maps_Link = f"https://www.google.com/maps/search/?api=1&query={encoded_address}" if encoded_address else 'N/A'

        return {
            'Property Name': prop_name.text.strip() if prop_name else 'N/A',
            'Address': address.text.strip() if address else 'N/A',
            'Area': area_tag.text.strip() if area_tag else 'N/A',
            'Rental price': rental_price.text.strip() if rental_price else 'N/A',
            'Property Type': property_type.text.strip() if property_type else 'N/A',
            'Carpet area(sq ft)': carpet_area.text.strip() if carpet_area else 'N/A',
            'Contact': Contact,
            'Maps link': Maps_Link,
            'url': url
        }

    except Exception as e:
        logging.error(f"Error scraping {url}: {e}")
        return None

# Main function
def main():
    logging.info("Starting listing extraction...")
    links = get_listing_links(pages=2)  
    logging.info(f"Collected {len(links)} property links.")

    browser = init_browser()
    results = []

    for idx, link in enumerate(links):
        logging.info(f"Scraping {idx + 1}/{len(links)}: {link}")
        data = extract_property_data(browser, link)
        if data:
            results.append(data)
        sleep(random.uniform(2, 6))

    browser.quit()

    if results:
        df = pd.DataFrame(results)
        df.to_csv('99acres_commercial_rent_contactno.csv', index=False)
        logging.info("Data saved to 99acres_commercial_rent_contactno.csv")
    else:
        logging.warning("No data extracted.")

if __name__ == "__main__":
    main()


2025-08-01 11:33:48,161 - INFO - Starting listing extraction...
2025-08-01 11:33:48,165 - INFO - Fetching page 1
2025-08-01 11:33:52,266 - INFO - Fetching page 2
2025-08-01 11:34:07,430 - WARNING - Failed on page 2: HTTPSConnectionPool(host='www.99acres.com', port=443): Read timed out. (read timeout=15)
2025-08-01 11:34:07,434 - INFO - Collected 31 property links.
2025-08-01 11:34:08,946 - INFO - Scraping 1/31: https://www.99acres.com/ready-to-move-office-space-for-rent-lease-in-bannerghatta-road-bangalore-south-5000-sq-ft-r1-spid-X78212991
2025-08-01 11:34:25,109 - WARNING - Couldn't click Contact Dealer on https://www.99acres.com/ready-to-move-office-space-for-rent-lease-in-bannerghatta-road-bangalore-south-5000-sq-ft-r1-spid-X78212991: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff7c0dce935+77845]
	GetHandleVerifier [0x0x7ff7c0dce990+77936]
	(No symbol) [0x0x7ff7c0b89cda]
	(No symbol) [0x0x7ff7c0be06aa]
	(No symbol) [0x0x7ff7c0be095c]
	(No symbol) [0x0x7ff7c0c33d07]
	(No symbol) 